In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import arff
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout, Activation
from sklearn.preprocessing import Normalizer
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [82]:
path="C:/Users/HP/Documents/CyberProject/"
dataOrignal=pd.read_csv(path+"UNR-IDD.csv")

In [83]:
dataOrignal.head()

,Switch ID,Port Number,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Packets Rx Dropped,Packets Tx Dropped,Packets Rx Errors,...,Unknown Load/Latest,Latest bytes counter,is_valid,Table ID,Active Flow Entries,Packets Looked Up,Packets Matched,Max Size,Label,Binary Label
0,of:000000000000000c,Port#:1,132,9181,6311853,238,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
1,of:000000000000000c,Port#:2,187,6304498,15713,171,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
2,of:000000000000000c,Port#:3,235,6311567,8030,58,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
3,of:000000000000000c,Port#:4,59,7878,16439,182,46,0,0,0,...,0,0,True,0,9,767,688,-1,TCP-SYN,Attack
4,of:000000000000000a,Port#:1,188,6304547,16497,183,46,0,0,0,...,0,0,True,0,7,489,403,-1,TCP-SYN,Attack


In [99]:
data= dataOrignal.apply(lambda x: pd.Categorical(x).codes if x.dtype == 'object' else x)

In [100]:
data.head()

,Switch ID,Port Number,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Packets Rx Dropped,Packets Tx Dropped,Packets Rx Errors,...,Unknown Load/Latest,Latest bytes counter,is_valid,Table ID,Active Flow Entries,Packets Looked Up,Packets Matched,Max Size,Label,Binary Label
0,11,0,132,9181,6311853,238,46,0,0,0,...,0,0,True,0,9,767,688,-1,5,0
1,11,1,187,6304498,15713,171,46,0,0,0,...,0,0,True,0,9,767,688,-1,5,0
2,11,2,235,6311567,8030,58,46,0,0,0,...,0,0,True,0,9,767,688,-1,5,0
3,11,3,59,7878,16439,182,46,0,0,0,...,0,0,True,0,9,767,688,-1,5,0
4,9,0,188,6304547,16497,183,46,0,0,0,...,0,0,True,0,7,489,403,-1,5,0


In [101]:
data.var()

Switch ID                        1.421493e+01
Port Number                      1.030540e+00
Received Packets                 4.261892e+09
Received Bytes                   1.371254e+15
Sent Bytes                       1.183275e+15
Sent Packets                     7.884192e+09
Port alive Duration (S)          9.645468e+05
Packets Rx Dropped               0.000000e+00
Packets Tx Dropped               0.000000e+00
Packets Rx Errors                0.000000e+00
Packets Tx Errors                0.000000e+00
Delta Received Packets           1.181875e+06
Delta Received Bytes             1.447523e+12
Delta Sent Bytes                 1.332209e+12
Delta Sent Packets               1.066124e+06
Delta Port alive Duration (S)    1.323999e-01
Delta Packets Rx Dropped         0.000000e+00
 Delta Packets Tx Dropped        0.000000e+00
Delta Packets Rx Errors          0.000000e+00
Delta Packets Tx Errors          0.000000e+00
Connection Point                 1.322486e+00
Total Load/Rate                  1

In [115]:
data.describe()

,Port Number,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Delta Received Packets,Delta Received Bytes,Delta Sent Bytes,Delta Sent Packets,...,Unknown Load/Rate,Unknown Load/Latest,Latest bytes counter,Table ID,Active Flow Entries,Packets Looked Up,Packets Matched,Max Size,Label,Binary Label
count,37411.000000,37411.000000,3.741100e+04,3.741100e+04,37411.000000,37411.000000,37411.000000,3.741100e+04,3.741100e+04,37411.000000,...,3.741100e+04,3.741100e+04,3.741100e+04,37411.0,37411.000000,3.741100e+04,3.741100e+04,37411.0,37411.000000,37411.000000
mean,1.172676,21618.897169,2.647491e+07,2.451212e+07,33626.824009,911.096415,194.958087,3.427001e+05,3.145126e+05,178.083264,...,1.921455e+04,5.145597e+05,1.921455e+04,0.0,88.690572,1.008636e+05,1.007444e+05,-1.0,2.663174,0.100853
std,1.015155,65283.170126,3.703044e+07,3.439876e+07,88792.970378,982.113446,1087.140797,1.203131e+06,1.154213e+06,1032.533005,...,1.107441e+05,1.863403e+06,1.107441e+05,0.0,790.715343,2.350297e+05,2.350284e+05,0.0,1.959171,0.301138
min,0.000000,9.000000,7.860000e+02,5.705000e+03,41.000000,26.000000,0.000000,0.000000e+00,2.780000e+02,2.000000,...,-1.284277e+06,0.000000e+00,-1.284277e+06,0.0,4.000000,8.700000e+01,3.700000e+01,-1.0,0.000000,0.000000
25%,0.000000,329.000000,9.104050e+04,5.775950e+04,347.000000,136.000000,2.000000,2.780000e+02,2.800000e+02,2.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,5.000000,2.367000e+03,2.272000e+03,-1.0,1.000000,0.000000
50%,1.000000,1170.000000,1.263052e+07,1.262658e+07,1240.000000,259.000000,4.000000,5.560000e+02,5.560000e+02,4.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,6.000000,7.472000e+03,7.349000e+03,-1.0,3.000000,0.000000
75%,2.000000,3417.000000,3.783230e+07,3.176443e+07,3968.000000,1747.000000,6.000000,8.030000e+02,7.590000e+02,5.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,8.000000,2.343700e+04,2.322000e+04,-1.0,4.000000,0.000000
max,3.000000,352772.000000,2.715925e+08,2.392430e+08,421598.000000,3317.000000,15695.000000,6.647966e+06,6.647970e+06,15659.000000,...,2.004388e+06,2.606759e+07,2.004388e+06,0.0,9987.000000,1.012574e+06,1.012439e+06,-1.0,5.000000,1.000000


In [102]:
#Columns with 0 variance
to_drop_Columns1= ['Delta Packets Rx Dropped', 
'Delta Packets Rx Errors',
'Delta Packets Tx Errors','Packets Rx Dropped',
'Packets Tx Dropped',             
'Packets Rx Errors',               
'Packets Tx Errors' , 'Switch ID'  ]

In [103]:
data=data.drop(to_drop_Columns1,axis=1)

MULTILABEL CLASSIFICATION

In [104]:
#Separate X and Y
X=data.drop('Label',axis=1)
# Y is the attack column which specifies what type of attack is 
y=data['Label']

In [105]:
print(X.shape)

(37411, 25)


In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [107]:
data['Label'].unique()
print(len(data['Label'].unique()))

6


In [108]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on test set and calculate accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.9218227983429106


In [42]:
scaler = Normalizer().fit(X)
normalized_X = scaler.transform(X)
normalized_X.shape
y_onehot = to_categorical(y, num_classes=6)

In [43]:
X_normalized_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_normalized_reshaped, y_onehot, test_size=0.2, random_state=42)
y_onehot.shape

(37411, 6)

In [39]:
X_normalized_reshaped.shape

(37411, 1, 25)

Binary Classification - Anamoly Classification

In [109]:
X2=data.drop(['Binary Label','Label'],axis=1)
# Y is the attack column which specifies what type of attack is 
y2=data['Binary Label']

In [110]:
X2.head()

,Port Number,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Delta Received Packets,Delta Received Bytes,Delta Sent Bytes,Delta Sent Packets,...,Total Load/Latest,Unknown Load/Rate,Unknown Load/Latest,Latest bytes counter,is_valid,Table ID,Active Flow Entries,Packets Looked Up,Packets Matched,Max Size
0,0,132,9181,6311853,238,46,0,0,280,2,...,0,0,0,0,True,0,9,767,688,-1
1,1,187,6304498,15713,171,46,146,5908166,5969,84,...,0,0,0,0,True,0,9,767,688,-1
2,2,235,6311567,8030,58,46,2,278,280,2,...,0,0,0,0,True,0,9,767,688,-1
3,3,59,7878,16439,182,46,2,278,280,2,...,0,0,0,0,True,0,9,767,688,-1
4,0,188,6304547,16497,183,46,0,0,280,2,...,0,0,0,0,True,0,7,489,403,-1


In [111]:
y2.head()

0    0
1    0
2    0
3    0
4    0
Name: Binary Label, dtype: int8

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [113]:
clf2 = DecisionTreeClassifier(random_state=42)
clf2.fit(X_train, y_train)

# Predict on test set and calculate accuracy
y_pred = clf2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  1.0
